In [ ]:
# 1. Preparing corpus - 뉴스기사 크롤링 (네이버링크가 있는 신문사 - 연합뉴스, 이데일리)

import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
def naver_news_crawling(query, start_date, end_date, s_from, e_to, page):
    title_list=[]
    date_list=[]
    contents_list=[]
    #최대5페이지까지
    while page != 0:
        url="https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+query+"&sort=0&photo=0&field=0&pd=3&ds="+start_date+"&de="+ end_date +"&cluster_rank=10&mynews=1&office_type=2&office_section_code=8&news_office_checked=1001&nso=so:r,p:from"+s_from+"to"+e_to+",a:all&start="+str(page)+"&refresh_start=0"
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        req = requests.get(url,headers=header)
        cont = req.text
        soup = BeautifulSoup(cont, 'html.parser')
        for urls in soup.select("a.info"):
            try :
                if urls["href"].startswith("https://news.naver.com"):
                    title_text, date_text, contents_text= get_news(urls["href"])  
                    title_list.append(title_text)
                    date_list.append(date_text)
                    contents_list.append(contents_text)
            except Exception as e:
                print(e) 
                continue

In [ ]:
news_company = {'연합뉴스':'1001','이데일리':'2227'}
news_company['연합뉴스']

'1001'

In [ ]:
# 1. Preparing corpus - 뉴스기사 크롤링 (네이버링크가 있는 신문사 - 연합뉴스, 이데일리)

import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

def get_news(url):      
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110                                   Safari/537.36'}
    html = requests.get(url, headers=header)
    bsoup = BeautifulSoup(html.content, 'html.parser')    
    # 기사 제목
    title = bsoup.select('h3#articleTitle')[0].text
    # 날짜 
    pdate = bsoup.select('.t11')[0].get_text()[:10]
    # 기사 본문
    content = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    ctext = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "") 
    return title, pdate, ctext

def naver_news_crawling(query, start_date, end_date, s_from, e_to, page):
    
    title_list=[]
    date_list=[]
    contents_list=[]
    
    # 언론사에 따라 url주소의 언론사 숫자 변경
    news_company = {'연합뉴스':'1001','이데일리':'2227'}
    while page != 0:
  
        url="https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+query+"&sort=0&photo=0&field=0&pd=3&ds="+start_date
        +"&de="+ end_date +"&cluster_rank=10&mynews=1&office_type=2&office_section_code=8&"
        +"news_office_checked="+news_company['연합뉴스']+"&nso=so:r,p:from"+s_from+"to"+e_to+",a:all&start="+str(page)+"&refresh_start=0"
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110                                   Safari/537.36'}
        req = requests.get(url,headers=header)
        cont = req.text
        soup = BeautifulSoup(cont, 'html.parser')

        for urls in soup.select("a.info"):
            try :


                if urls["href"].startswith("https://news.naver.com"):
                   
                    title_text, date_text, contents_text= get_news(urls["href"])
                    
                    title_list.append(title_text)
                    date_list.append(date_text)
                    contents_list.append(contents_text)
                    
            except Exception as e:
                print(e) 
                continue

        # 네이버 검색결과의 다음 페이지 버튼을 가져온다.
        btn_next = soup.select_one('#main_pack > div.api_sc_page_wrap > div > a.btn_next')
        # 다음 페이지 버튼이 없으면(검색 결과 없을 경우) 크롤링 하지 않고 다음 날짜 링크로 가기
        if btn_next == None:
            break
        
        # 페이지의 다음 버튼이 비활성화 될때(마지막 페이지)까지 페이지 넘기기
        is_last = btn_next['aria-disabled']
        # 마지막 페이지일 경우 다음 날짜로 가기 위해 while 종료
        if is_last == "true":
            break
        # 네이버 페이지가 1, 11, 21, 31 순으로 가기 때문에 10씩 더해준다.
        else:
            page += 10

    # 데이터 프레임 형식으로 만들어 데이터 반환   
    result={}   
    result= {'date' : date_list, 'title':title_list,'contents': contents_list}
    
    df=[]
    df=pd.DataFrame(result)
    
    return df

error_date=[]

page = 1
query = "금리" 
date = datetime.date(2021,1,1) #시작날짜
cur_date=date.strftime('%Y.%m.%d')
e_to = s_from = cur_date.replace(".","")
while(cur_date!="2022.01.01"):#끝날짜 
    try:
        print(""+cur_date+"날짜의 크롤링을 시작합니다")
        new_df = naver_news_crawling(query, cur_date, cur_date, s_from, e_to, page)
        new_df=new_df.drop_duplicates()

        if len(new_df) > 0:
            new_df['title'] = new_df['title'].str.encode('utf-8', 'ignore').str.decode('utf-8')
            new_df['contents'] = new_df['contents'].str.encode('utf-8', 'ignore').str.decode('utf-8')
            new_df.to_csv(r"금리2021.csv",encoding='utf-8-sig',index=False, header=False, mode='a')
         
        date += datetime.timedelta(days=1)
        cur_date=date.strftime('%Y.%m.%d')
        e_to = s_from = cur_date.replace(".","")
        print("성공")
    except Exception as e:
        error_date.append(cur_date)
        print(e)
        print("실패")
        continue
    
print("크롤링을 종료합니다")

In [ ]:
# 1. Preparing corpus - 뉴스기사 크롤링 (네이버링크가 없는 신문사 - 연합인포맥스)
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

titles = []
dates = []
contents = []
first_page = 1
last_page = 1001

for i in tqdm(range(first_page,last_page)):
    urls = []
    url = 'https://news.einfomax.co.kr/news/articleList.html?page='+str(i)+'&total=157314&box_idxno=&sc_area=A&view_type=sm&sc_section_code=&sc_level=&sc_article_type=&sc_sdate=2008-01-01&sc_edate=2021-12-31&sc_order_by=E&sc_word=%EA%B8%88%EB%A6%AC&sc_andor=OR&sc_word2='
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news = soup.select('section[class="article-list-content type-sm text-left"] div[class="list-block"]')

    for i in range(len(news)):
        data = soup.select_one('#user-container > div.float-center.max-width-1080 > div.user-content.list-wrap > section > article > div.article-list > section > div:nth-of-type('+str(+i+1)+') > p > a')
        urls.append('https://news.einfomax.co.kr'+(data['href']))

    for href in urls:
        response = requests.get(href)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.select_one('div[class="article-head-title"]')
        date= soup.select('div[class="info-text"] li')
        s = date[1].text
        content =  soup.select_one('#article-view-content-div')
        # 기사 날짜 정보 리스트에 append
        dates.append(s[4:15])
        # 기사 제목 정보 리스트에 append
        titles.append(title.text)
        # 기사 내용 정보 리스트에 append
        contents.append(content.text)

news_df = pd.DataFrame({'title':titles, 'date':dates,'content':contents})
news_df.to_csv(f'인포맥스{first_page}-{last_page}.csv',encoding='utf-8-sig',index=False)